# Lab | Data Cleaning and Formatting

# 01 - Problem

## Case study

* Data Description.
* Goal

In [60]:
#LAB W.1 
#customer data from an insurance company: https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

# 02 - Getting Data

* Read the .csv file
* Import what's needed

In [61]:
import pandas as pd
import numpy as np
df= pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')


In [62]:
def cleaning_df(df):
    new_cols = []
    
    for i in range(len(df.columns)):
        new_cols.append(df.columns[i].lower().replace(' ', '_'))  # Underscoring
        
    df.columns = new_cols
    df = df.rename(columns={'st': 'state'})
    df = df.drop_duplicates()
    
    # df = df.drop(1071, axis=0)  # Full NaN row
    
    df['gender'] = df['gender'].fillna('F')  # We choose Mode Female
    df['gender'] = np.where(df['gender'].isin(['F', 'female', 'Femal']), 'F', 'M')
    
    # Removing % in customer value
    df['customer_lifetime_value'] = df['customer_lifetime_value'].str.replace('%', '')
    
    # Convert to dollars and turn into numeric
    df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce') / 100
    
    # Removing the slash ('/')
    no_slash_list = []
    for item in df['number_of_open_complaints']:
        if pd.notna(item):  # Check for NaN before splitting
            no_slash_list.append(item.split('/')[1] if '/' in item else item)
        else:
            no_slash_list.append(item)
    df['number_of_open_complaints'] = no_slash_list
    
    # Turn all to_numeric()
    df['number_of_open_complaints'] = pd.to_numeric(df['number_of_open_complaints'], errors='coerce')
    df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')
    
    return df
    
        # let's test it
df=cleaning_df(df)

# 03 - Cleaning Data

* headers names.
* NaN and duplicates.
* Categorical Features.
* Numerical Features.
* Exploration.

In [63]:
#Exploring
df.dtypes
#df.columns

customer                      object
state                         object
gender                        object
education                     object
customer_lifetime_value      float64
income                       float64
monthly_premium_auto         float64
number_of_open_complaints    float64
policy_type                   object
vehicle_class                 object
total_claim_amount           float64
dtype: object

In [64]:
#removing duplicates
df.drop_duplicates(keep=False)
#column names
cols=[]
for colname in df.columns:
    cols.append(colname.lower())
df.columns = cols

df.columns=df.columns.str.replace(' ','_')
#df['state']=df.column.str.replace('st','state')
df.rename(columns={'st':'state'},inplace=True) 
#inplace=True: assign the result back, as the modification is not-inplace
print(df.columns)




Index(['customer', 'state', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'policy_type', 'vehicle_class', 'total_claim_amount'],
      dtype='object')


In [65]:
df.shape
df['gender'].unique()
df['gender'].value_counts()
## value counts(dropna=False) tells how many NaN there is

gender
F    620
M    452
Name: count, dtype: int64

In [66]:
#removing the slash using split('/')
#no_slash_list=[]
#for item in df['number_of_open_complaints']: 
#    no_slash_list.append(item.split('/')[1]) #(index1) keeps middle number 
# have created a new list
#df['number_of_open_complaints']=no_slash_list


In [67]:
# cleaning female male
def cleangender(x):
    M_target=['Male']
    F_target=['Femal', 'female']
    
    if x in M_target:
        return 'M'
    elif x in F_target:
        return 'F'
    else:
        return x

df['gender'] = list(map(cleangender, df['gender']))
df['gender'].unique()
df['gender'].value_counts()
##FYI startswith() doesnt work  
##error 'float' object has no attribute 'startswith'
##other way around: np.where()isin[](see bonus 1)

gender
F    620
M    452
Name: count, dtype: int64

In [68]:
df['state'].unique()
# cleaning states
def cleanstate(x):
    if x == 'WA':
        return 'Washington'
    elif x =='AZ':
        return 'Arizona'
    elif x=='Cali':
        return 'California'
    else:
        return x

df['state'] = list(map(cleanstate, df['state']))
df['state'].value_counts()


##other way around: using dictionaries {}

state
California    331
Oregon        320
Arizona       211
Washington    111
Nevada         98
Name: count, dtype: int64

In [69]:
#cleaning education
df['education']=df['education'].str.replace('Bachelors', 'Bachelor')
df['education'].value_counts()

education
Bachelor                331
College                 313
High School or Below    296
Master                   94
Doctor                   37
Name: count, dtype: int64

In [70]:
# cleaning car class
def cleancarclass(x):
    luxury_target=['Luxury SUV','Luxury Car','Sports Car']
    if x in luxury_target:
        return 'Luxury'
    else:
        return x

df['vehicle_class'] = list(map(cleancarclass, df['vehicle_class']))
df['vehicle_class'].value_counts()

vehicle_class
Four-Door Car    576
Two-Door Car     205
SUV              199
Luxury            91
Name: count, dtype: int64

In [71]:
# removing % in customer value 
#df['customer_lifetime_value']=df['customer_lifetime_value'].str.replace('%', '')
# convert to dollars and turn into numeric
#df['customer_lifetime_value']=pd.to_numeric(df['customer_lifetime_value'], errors='coerce')/100


In [72]:
# turn new list to_numeric()
df['number_of_open_complaints']=pd.to_numeric(df['number_of_open_complaints'],errors='coerce')

In [73]:
# let's see the percentage of null values in a column
#df.isnull()
#round(df.isna().sum()/len(df),4)*100  
#nulls_df = pd.df(round(df.isna().sum()/len(df),4)*100)
#nulls_df.columns = ['header_name', 'percent_nulls']
#nulls_df

# 04 - Processing Data

* Dealing with outliers.
* Normalization.
* Encoding Categorical Data.
* Splitting into train set and test set.

* Apply model.

In [74]:
df=df.drop_duplicates() #we assume duplicates are those with NaN in all columns
 #previous command drops everything except the first duplicate
df=df.drop(1071,axis=0,inplace=True)

In [75]:
#df[df['state'].isnull()==True]
#dropna shows how many nulls there are also
#df['state'].value_counts(dropna=False) 


In [76]:
#MISSING VALUES
#CATEGORICAL we fill with the Mode which is Oregon
df['state']=df['state'].fillna('Oregon')

#ORDINAL we fill with the mean
mean_income=df['income'].mean()
mean_income
df['income']=df['income'].fillna(mean_income)
df.isna().sum #check

#lets remove the decimals with round

TypeError: 'NoneType' object is not subscriptable

In [ ]:
## adding index=False (you dont get unamed 0 column when opening it again)

In [ ]:
##pd.to_numeric(df['number_of_open_complaints'], errors='coerce')

# 05 - Modeling

In [77]:
#install
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [80]:

#Show a plot of the Gender breakdown.
sns.countplot('gender')
plt.ylabel('Gender Composition') #adding a title. thanks Sandra!
#Conclusion: more Female clients than Male clients

#Show a plot of Income by State.
sns.barplot(x="state",y="income",data=df)
plt.ylabel('Income by state')
plt.show()
#Conclusion: clients in Washington have the highest income averages

ValueError: could not convert string to float: 'gender'

In [ ]:
#Show a plot of the Policy Type by the Total Claim Amount.
sns.barplot(x="policy_type", x="total_claim_amount",data=df
plt.show()


In [ ]:
#Show a plot of the Education by Vehicle Class. both are categorical. 
sns.histplot(x="education", hue="vehicle_class",data=df, multiple='stack')
plt.show()


In [ ]:
#hue gives the other one as hue. barplot needs numeric so we do hist
sns.histplot(x="education", hue="vehicle_class",data=df1,multiple='stack')
#multiple = stack : separates the colours by stacking
plt.show()
#looks like people with least 

In [ ]:

#some skewed to the right
#if anything is multicollinear we would drop it; but there isnt
#negative influence of the income

# 06 - Model Validation
 

Scaling (numerical)

In [81]:
#splitting Y (our target) and X (ALL our features)
y = df['total_claim_amount']
X=df.drop(['total_claim_amount'], axis=1) 
#all the rest. iloc could also be used

#'customers' is useless so drop it too

X_num=X.select_dtypes(include = np.number)
X_cat=X.select_dtype(include=object) #that would be :
#'customer', 'state', 'gender', 'education','policy_type', 'vehicle_class'

##SCALING##
# Normalize using minmaxscaler
transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
print(x_normalized.shape)
x_normalized
datanorm = pd.DataFrame(x_normalized, columns=X_num.columns) 

# Plot the distributions again.
for i in range(3):
    sns.distplot(datanorm.iloc[:,i])
    plt.show()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(X_cat)
cols = encoder.get_feature_names(input_features=X_cat.columns)
X_cat_encode = pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)
X_cat_encode.head()

X=pd.concat([X_num_scale, X_cat_encode], axis=1) 


In [ ]:
sns.countplot(data= x_normalized,x='monthly_premium_auto')

In [ ]:
correlations = x_normalized.corr()
correlations

Train test split

In [ ]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test=tts(X, y, test_size=.2)

In [ ]:
from sklearn.linear_model import LinearRegression as linreg

lm = linreg()    # model
lm.fit(X_train, y_train)   # model training
y_pred = lm.predict(X_test)   # model prediction


* R2.
* MSE.
* RMSE.
* MAE.

In [ ]:
#R2
print ('train R2: {} -- test R2: {}'.format(lm.score(X_train, y_train),
                                            lm.score(X_test, y_test)))

In [ ]:
#MSE
from sklearn.metrics import mean_squared_error as mse


train_mse=mse(lm.predict(X_train), y_train)
test_mse=mse(y_pred, y_test)

print ('train MSE: {} -- test MSE: {}'.format(train_mse, test_mse))

# 07 - Reporting

* Present results.